# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

file_path = os.path.join('output_data','cities.csv')
cities_df = pd.read_csv(file_path)
cities_df.head()

,index,City,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness,Country
0,0,hithadhoo,-0.6000,73.0833,84.07,84.07,71,7.31,72,MV
1,1,hami,42.8000,93.4500,75.00,75.00,13,7.63,2,CN
2,2,qui nhon,13.7667,109.2333,82.45,82.45,68,5.41,66,VN
3,3,cidreira,-30.1811,-50.2056,65.14,65.14,78,6.87,1,BR
4,4,khatanga,71.9667,102.5000,32.20,32.20,93,6.96,100,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)
humidity.max()

100.0

In [4]:
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity'].astype(float)
fig = gmaps.figure(zoom_level=3, center=((0,0)))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
max_temp = 80
min_temp = 50
wind_speed = 10
cloudiness = 0


cities_max_temp = cities_df.loc[cities_df['Temperature'] <= max_temp,:]
cities_max_temp
cities_min_temp = cities_max_temp.loc[cities_max_temp['Temperature']>min_temp,:]
cities_min_temp
cities_wind = cities_min_temp.loc[cities_min_temp['Wind Speed (mph)']<wind_speed,:]
cities_wind
cities_map = cities_wind.loc[cities_wind['Cloudiness']==cloudiness,:]

cities_map.reset_index(drop=True, inplace=True)
cities_map.head()

,index,City,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness,Country
0,23,fram,-26.9833,-55.8833,64.40,64.40,59,4.61,0,PY
1,41,pisco,-13.7000,-76.2167,66.11,66.20,88,8.05,0,PE
2,45,virden,39.5009,-89.7679,64.65,69.01,75,5.01,0,US
3,47,cayenne,4.9333,-52.3333,76.24,77.00,94,2.48,0,GF
4,58,mar del plata,-38.0023,-57.5575,61.30,62.01,82,6.91,0,AR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

#locations = cities_map[["Latitude", "Longitude"]]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "nearest hotel",
    "key": g_key
}


# use iterrows to iterate through pandas dataframe
for index, row in cities_map.iterrows():

    # get location
    city_name = row['City']
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    #print(response)
    # extract results
    results = response['results']
   
    
    try:
        print(f"Closest {city_name} Hotel is {results[0]['name']}.")

        cities_map.loc[index,'Hotel Name'] =results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: fram.
Closest fram Hotel is Carnaval Hotel Casino.
------------
Retrieving Results for Index 1: pisco.


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest pisco Hotel is Hotel Las Dunas.
------------
Retrieving Results for Index 2: virden.
Closest virden Hotel is Holiday Inn Express & Suites Jacksonville.
------------
Retrieving Results for Index 3: cayenne.
Closest cayenne Hotel is Grand Hotel Montabo.
------------
Retrieving Results for Index 4: mar del plata.
Closest mar del plata Hotel is Hermitage Hotel.
------------
Retrieving Results for Index 5: zhangye.
Closest zhangye Hotel is Jinyang International Hotel.
------------
Retrieving Results for Index 6: cape town.
Closest cape town Hotel is Atlanticview Cape Town Boutique Hotel.
------------
Retrieving Results for Index 7: villa carlos paz.
Closest villa carlos paz Hotel is Lake Buenavista Resort SPA.
------------
Retrieving Results for Index 8: hualmay.
Closest hualmay Hotel is Empedrada Lodge- Hotel Caral.
------------
Retrieving Results for Index 9: houma.
Closest houma Hotel is Days Inn by Wyndham Galliano LA.
------------
Retrieving Results for Index 10: chuy.
Closest 

In [7]:
hotel_df = cities_map.rename(columns={'Latitude': 'Lat','Longitude':'Lng'}).dropna()

hotel_df.head()

,index,City,Lat,Lng,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness,Country,Hotel Name
0,23,fram,-26.9833,-55.8833,64.40,64.40,59,4.61,0,PY,Carnaval Hotel Casino
1,41,pisco,-13.7000,-76.2167,66.11,66.20,88,8.05,0,PE,Hotel Las Dunas
2,45,virden,39.5009,-89.7679,64.65,69.01,75,5.01,0,US,Holiday Inn Express & Suites Jacksonville
3,47,cayenne,4.9333,-52.3333,76.24,77.00,94,2.48,0,GF,Grand Hotel Montabo
4,58,mar del plata,-38.0023,-57.5575,61.30,62.01,82,6.91,0,AR,Hermitage Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
#figure_layout = {
#    'width': '1000px',
#    'height': '800px',
#    'border': '1px solid black',
#    'padding': '1px',
#    'margin': '0 auto 0 auto'
#}




# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
#fig
fig

Figure(layout=FigureLayout(height='420px'))